## Preprocessing

In [38]:
# Import our dependencies
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [39]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
dropped_df = application_df.drop(columns=['EIN', 'NAME'])

# Display the updated DataFrame to verify the changes
print(dropped_df)

      APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0                  T10       Independent          C1000    ProductDev   
1                   T3       Independent          C2000  Preservation   
2                   T5  CompanySponsored          C3000    ProductDev   
3                   T3  CompanySponsored          C2000  Preservation   
4                   T3       Independent          C1000     Heathcare   
...                ...               ...            ...           ...   
34294               T4       Independent          C1000    ProductDev   
34295               T4  CompanySponsored          C3000    ProductDev   
34296               T3  CompanySponsored          C2000  Preservation   
34297               T5       Independent          C3000    ProductDev   
34298               T3       Independent          C1000  Preservation   

       ORGANIZATION  STATUS     INCOME_AMT SPECIAL_CONSIDERATIONS   ASK_AMT  \
0       Association       1              0  

In [40]:
# Determine the number of unique values in each column.
unique_values = dropped_df.nunique()

# Display the number of unique values for each column
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [41]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
# Step 1: Get the value counts for APPLICATION_TYPE
application_type_counts = dropped_df['APPLICATION_TYPE'].value_counts()

# Step 2: Define a threshold for infrequent application types
threshold = 10

# Step 3: Replace infrequent application types with 'Other'
dropped_df['APPLICATION_TYPE'] = dropped_df['APPLICATION_TYPE'].apply(lambda x: 'Other' if application_type_counts[x] < threshold else x)

# Display the updated DataFrame to verify the changes
print(dropped_df['APPLICATION_TYPE'].value_counts())


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: count, dtype: int64


In [42]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_type_counts[application_type_counts < threshold].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: count, dtype: int64

In [43]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = dropped_df['CLASSIFICATION'].value_counts()

In [44]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_filtered = classification_counts[classification_counts > 1]

In [45]:
# Choose a cutoff value and create a list of classifications to be replaced
cutoff_value = 10

# use the variable name `classifications_to_replace`
classifications_to_replace = classification_counts_filtered[classification_counts_filtered < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
Other       72
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C2561        1
C1580        1
C1900        1
C1820        1
C6100        1
C1248        1
C2600        1
C1370        1
C5200        1
C4200        1
C2190        1
C1236        1
C2570        1
C2500        1
C3700        1
C1570        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120        1
C8210        1
C4500        1
C1245        1
C2150        1
Name: count, dtype: int64

In [46]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = ['APPLICATION_TYPE', 'CLASSIFICATION', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'SPECIAL_CONSIDERATIONS']

# Convert categorical columns to numeric using pd.get_dummies
categorical_df = pd.get_dummies(dropped_df, columns=categorical_columns)

# Step 3: Display the updated DataFrame to verify the changes
print(categorical_df.head())

      INCOME_AMT  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0              0     5000              1                   False   
1         1-9999   108590              1                   False   
2              0     5000              0                   False   
3    10000-24999     6692              1                   False   
4  100000-499999   142590              1                   False   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T12  APPLICATION_TYPE_T13  \
0                  True                 False                 False   
1                 False                 False                 False   
2                 False                 False                 False   
3                 False                 False                 False   
4                 False                 False                 False   

   APPLICATION_TYPE_T19  APPLICATION_TYPE_T2  APPLICATION_TYPE_T3  ...  \
0                 False                False                False  ...   
1               

In [47]:
# Split our preprocessed data into our features and target arrays
# Assuming 'IS_SUCCESSFUL' is the target column and application_df is your preprocessed DataFrame
X = categorical_df.drop(columns=['IS_SUCCESSFUL'])  # Features (all columns except the target)
y = categorical_df['IS_SUCCESSFUL']  # Target column

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the split by checking the shapes of the resulting datasets
print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Training Target Shape:", y_train.shape)
print("Testing Target Shape:", y_test.shape)

Training Features Shape: (27439, 105)
Testing Features Shape: (6860, 105)
Training Target Shape: (27439,)
Testing Target Shape: (6860,)


In [48]:
from sklearn.model_selection import train_test_split

# Convert categorical variables to numeric
application_df = pd.get_dummies(application_df)

# Convert Range Strings to Numeric Values
def convert_range_to_numeric(value):
    if isinstance(value, str):
        # Handle cases with 'M' for million
        if 'M' in value:
            value = value.replace('M', '')
            if '-' in value:
                low, high = value.split('-')
                return (float(low) + float(high)) / 2 * 1e6
            elif '+' in value:
                value = value.replace('+', '')
                return float(value) * 1e6  # Treat '50+' as just 50M
            else:
                return float(value) * 1e6
        
        # Handle cases with 'K' for thousand
        elif 'K' in value:
            value = value.replace('K', '')
            if '-' in value:
                low, high = value.split('-')
                return (float(low) + float(high)) / 2 * 1e3
            elif '+' in value:
                value = value.replace('+', '')
                return float(value) * 1e3  # Treat '50+' as just 50K
            else:
                return float(value) * 1e3
        
        # Handle cases with '-' but no 'M' or 'K'
        elif '-' in value:
            low, high = value.split('-')
            return (float(low) + float(high)) / 2
        
        # Handle cases with '+' without 'M' or 'K'
        elif '+' in value:
            value = value.replace('+', '')
            return float(value)  # Treat '50+' as just 50

    # If not a string or does not match the expected formats, return the original value
    return value

# Ensure that the column exists and apply this function to the relevant columns
if 'INCOME_AMT' in application_df.columns:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].apply(convert_range_to_numeric)

# Split the data into features (X) and target (y)
X = application_df.drop(columns=['IS_SUCCESSFUL'])  # Features (all columns except the target)
y = application_df['IS_SUCCESSFUL']  # Target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler on the training data and scale the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [49]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

c:\Users\lexis\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 8)              │       157,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 5)              │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 157,387 (614.79 KB)

 Trainable params: 157,387 (614.79 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [51]:
# Define the callback to save the model's weights
checkpoint_path = "model_checkpoint.weights.h5"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 verbose=1)

# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7235 - loss: 0.6197
Epoch 2/100
 75/858 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8373 - loss: 0.3783

c:\Users\lexis\anaconda3\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8463 - loss: 0.3308
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9254 - loss: 0.1730
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9502 - loss: 0.1236
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9529 - loss: 0.1109
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9569 - loss: 0.1033
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9573 - loss: 0.1007
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9583 - loss: 0.0957
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9582 - loss: 0.0973
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9590 - loss: 0.0955
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9612 - loss: 0.0934
Epoch 12/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9586 - loss: 0.0929
Epoch 13/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/ste

In [52]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.6076 - loss: 0.6665
Loss: 0.6664589643478394, Accuracy: 0.6075801849365234


In [53]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

In [54]:
#Optimized Model 
# Optimize #1: Adding Dropout Layers
# Optimize #2: Early Stopping
# Optimize #3: Model Architecture Tuning

# Split the data into features and target
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a new model with dropout layers and early stopping
model = Sequential([
    Dense(80, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),  # Dropout layer to reduce overfitting
    Dense(50, activation='relu'),
    Dropout(0.2),  # Another dropout layer
    Dense(20, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer with sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=100, callbacks=[early_stopping], verbose=2)

# Save the model
model.save('AlphabetSoupCharity_optimized.h5')

c:\Users\lexis\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
858/858 - 14s - 16ms/step - accuracy: 0.7586 - loss: 0.5027 - val_accuracy: 0.6034 - val_loss: 0.5968
Epoch 2/100
858/858 - 9s - 10ms/step - accuracy: 0.8481 - loss: 0.3358 - val_accuracy: 0.5885 - val_loss: 0.6796
Epoch 3/100
858/858 - 9s - 10ms/step - accuracy: 0.9488 - loss: 0.1357 - val_accuracy: 0.6023 - val_loss: 0.6166
Epoch 4/100
858/858 - 9s - 10ms/step - accuracy: 0.9603 - loss: 0.1018 - val_accuracy: 0.5876 - val_loss: 0.6451
Epoch 5/100
858/858 - 9s - 11ms/step - accuracy: 0.9617 - loss: 0.0969 - val_accuracy: 0.5856 - val_loss: 0.6403
Epoch 6/100
858/858 - 10s - 12ms/step - accuracy: 0.9614 - loss: 0.0953 - val_accuracy: 0.5843 - val_loss: 0.6060
Epoch 7/100
858/858 - 9s - 10ms/step - accuracy: 0.9625 - loss: 0.0931 - val_accuracy: 0.5907 - val_loss: 0.6024
Epoch 8/100
858/858 - 11s - 13ms/step - accuracy: 0.9629 - loss: 0.0922 - val_accuracy: 0.5792 - val_loss: 0.6660
Epoch 9/100
858/858 - 13s - 15ms/step - accuracy: 0.9627 - loss: 0.0914 - val_accuracy: 0.583

Loss: 0.550747811794281, Accuracy: 0.6921283006668091


In [55]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.6076 - loss: 0.6665
Loss: 0.6664589643478394, Accuracy: 0.6075801849365234
